In [1]:
from dotenv import load_dotenv
import os
import random
import itertools
import asyncio
import openai
load_dotenv()
oai_key=os.getenv("OPENAI_KEY")
openai.api_key = oai_key
from concurrent.futures import ThreadPoolExecutor

True

In [79]:
async def call_model(prompt:str,max_tokens:int=256,temp:float=0):
    completion = openai.Completion.create(engine="text-davinci-002",temperature=temp,max_tokens=max_tokens, prompt=prompt)

    # print the completion
    return completion.choices[0].text.strip("\n")

prompt="What is 2+2?"
with ThreadPoolExecutor() as p:
    tasks=[call_model("Tell me a story about a flying potato"),
          call_model("Tell me a story about a flying car")]
    result=await asyncio.gather(*tasks)
    print(result)



["There once was a potato who dreamed of flying. Every day, it would watch the birds soaring through the sky and longed to join them. One day, it decided to take the leap and try to fly. Unfortunately, potatoes can't fly. The potato crashed to the ground and was never seen again.", 'A flying car is a vehicle that can fly and drive on the ground. It is a type of aircraft that is powered by both an airplane engine and a car engine. The first flying car was invented in 1917 by American inventor Charles Kettering.']


# Grid shape task (2x2)
This task consists on figuring out what shapes are intersected by a line traced between two points in a 2x2 grid. In a 2x2 grid, if we clarify that start and end shapes do not count as being intersected, it does not matter what shapes and where they are: in a 2x2 grid there cannot be any intersections.

In [80]:
colors=['red','blue','green']
shapes=['circle','triangle','square']
prompt_start="In a 2x2 grid there are the following objects, listed left to right, top to bottom: "
perms=[f'{i[0]} {i[1]}' for i in itertools.product(colors,shapes)]
# The answer in all cases is none
def get_prompt_2x2():
    this_shapes=random.sample(perms,4)
    shapes=', '.join([i for i in this_shapes])
    two_arbitrary_shapes=random.sample(this_shapes,2)
    two_shapes_prompt=f'the {two_arbitrary_shapes[0]} and the {two_arbitrary_shapes[1]}'
    full_prompt='Question: '+ prompt_start+shapes+". If I draw a straight line between "+two_shapes_prompt+", how many shapes does that line intersect? The shapes where the line start and end do not count as intersecting the line."
    return full_prompt

get_prompt_2x2()

'Question: In a 2x2 grid there are the following objects, listed left to right, top to bottom: green square, red circle, blue triangle, green circle. If I draw a straight line between the green circle and the blue triangle, how many shapes does that line intersect? The shapes where the line start and end do not count as intersecting the line.'

In [89]:
N_SAMPLES=10
TEMP=0.7
async def eval_model(prompt):
        def check_answer(result:str):
        """
        Answer is always zero in this case
        """
        zeros={"0","zero","none","no"}
        for valid_answer in zeros:
            if valid_answer in result.lower():
                return True
        return False
    model_result=await call_model(prompt,10,TEMP)
    is_model_correct=check_answer(model_result)
    return prompt,model_result,is_model_correct

In [90]:
# 0% success rate
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_prompt_2x2()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[]

In [91]:
# Oneshot 0/50
prompt=lambda:get_prompt_2x2()+'\n\nAnswer: Zero\n\n'+get_prompt_2x2()+'\n\nAnswer:'
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(prompt()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[]

In [92]:
# Twoshot 2/50
prompt=lambda: get_prompt_2x2()+'\n\nAnswer: Zero\n\n'+get_prompt_2x2()+'\n\nAnswer: Zero\n\n'+get_prompt_2x2()+'\n\nAnswer:'
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(prompt()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[('Question: In a 2x2 grid there are the following objects, listed left to right, top to bottom: green triangle, blue circle, red triangle, green square. If I draw a straight line between the green triangle and the blue circle, how many shapes does that line intersect? The shapes where the line start and end do not count as intersecting the line.\n\nAnswer: Zero\n\nQuestion: In a 2x2 grid there are the following objects, listed left to right, top to bottom: green square, blue square, blue triangle, blue circle. If I draw a straight line between the blue square and the blue circle, how many shapes does that line intersect? The shapes where the line start and end do not count as intersecting the line.\n\nAnswer: Zero\n\nQuestion: In a 2x2 grid there are the following objects, listed left to right, top to bottom: red square, green circle, green square, red triangle. If I draw a straight line between the red triangle and the red square, how many shapes does that line intersect? The shapes 

# Grid shape task (3x3)
This is the same task as ealier, but on a 2d grid. Unlike th2 2x2 task, in the 3x3 task it is possible for there to be shapes intersecting the line if it is a diagonal line; there can be either zero or one shapes intersected. In this task, the model has to say not only whether there is a shape, but if there is, what the shape is.

In [274]:
prompt_start="In a 3x3 grid there are the following objects, listed left to right, top to bottom: "
intersecting_cells=[(1,3),(1,7),(1,9),(2,8),(3,7),(3,9),(4,6),(7,9)]
def get_prompt_3x3():
    this_shapes=random.sample(perms,9)
    shapes=', '.join([i for i in this_shapes])
    two_arbitrary_shapes=random.choice(intersecting_cells)
    shape_start=this_shapes[two_arbitrary_shapes[0]-1]
    shape_end=this_shapes[two_arbitrary_shapes[1]-1]
    two_shapes_prompt=f'the {shape_start} and the {shape_end}'
    full_prompt='Question: '+ prompt_start+shapes+\
    f". If I draw a straight line between {two_shapes_prompt}"+\
    ", what shapes, if any, does that line intersect?\
 The shapes where the line start and end do not count as intersecting the line.\
 If a shape is intersected, return the shape.\
    \nAnswer: "
    # Is there a shape that is intersected?
    intersected_shape=get_intersected_shape(this_shapes,shape_start,shape_end)
    return full_prompt,intersected_shape
def get_intersected_shape(shapes:list[str],shape1:str,shape2:str)->str|None:
    """
    1,2,3
    4,5,6
    7,8,9
    """
    pos1 = shapes.index(shape1)
    pos2=shapes.index(shape2)
    distance=abs(pos1-pos2)
    minpos=min(pos1,pos2)
    if distance in {8,4}:
        # diagonal
        return shapes[5]
    elif distance==2:
        # across row
        return shapes[minpos+1]
    elif distance==6:
        # across column
        return shapes[minpos+3]
    else:
        raise Exception
    
print(get_prompt_3x3())

('Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: green circle, blue circle, blue triangle, green triangle, red square, green square, red triangle, red circle, blue square. If I draw a straight line between the blue triangle and the blue square, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: ', 'green square')


In [277]:
N_SAMPLES=50
TEMP=0
sample_answer='The line intersects the green square'
def check_answer(result:str,answer:str):
      if str(answer) in result and len(result)<(len(sample_answer)-10):
            return True
      else:
        return False
# Assumes the number is the last part of the prompt
async def eval_model(prompt_and_answer:tuple[str,int]):
    prompt=prompt_and_answer[0]
    model_result=await call_model(prompt,1000,TEMP)
    is_model_correct=check_answer(model_result,prompt_and_answer[1])
    return prompt,model_result,is_model_correct
await eval_model(get_prompt_3x3())

('Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: green square, red circle, green triangle, blue triangle, blue circle, red triangle, green circle, red square, blue square. If I draw a straight line between the green triangle and the green circle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: ',
 'The line intersects the green square, the red circle, the blue triangle, the blue circle, and the red triangle.',
 False)

In [278]:
# Zeroshot: 0% success rate
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_prompt_3x3()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[]

In [289]:
# One-shot 2/50
N_SAMPLES=50
def get_example():
    prompt,answer=get_prompt_3x3()
    example=f"{prompt}{answer}\n"
    question=get_prompt_3x3()
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[('Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: blue square, blue circle, green square, green circle, red circle, red triangle, green triangle, red square, blue triangle. If I draw a straight line between the green square and the green triangle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: red triangle\n\nQuestion: In a 3x3 grid there are the following objects, listed left to right, top to bottom: red square, blue circle, green circle, red circle, green square, blue triangle, green triangle, blue square, red triangle. If I draw a straight line between the green triangle and the red triangle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: ',
  ' blue square',
  True),

In [310]:
def get_example():
    prompt,answer=get_prompt_3x3()
    prompt1,answer1=get_prompt_3x3()
    example=f"{prompt}{answer}\n"
    example1=f"{prompt1}{answer1}\n"
    example=f'{example}\n{example1}'
    question=get_prompt_3x3()
    return example+"\n"+question[0],question[1]
print(get_example()[0])

Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: red square, red circle, green circle, red triangle, blue triangle, blue circle, green triangle, blue square, green square. If I draw a straight line between the red square and the green triangle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    
Answer: red triangle

Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: red circle, red triangle, blue triangle, green square, green triangle, blue circle, blue square, red square, green circle. If I draw a straight line between the green square and the blue circle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    
Answer: green triangle

Question: In a 3x3 grid t

In [301]:
# Two-shot 5/50
N_SAMPLES=50
def get_example():
    prompt,answer=get_prompt_3x3()
    prompt1,answer1=get_prompt_3x3()
    example=f"{prompt}{answer}\n"
    example1=f"{prompt1}{answer1}\n"
    example=f'{example}\n{example1}'
    question=get_prompt_3x3()
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[('Question: In a 3x3 grid there are the following objects, listed left to right, top to bottom: red triangle, red circle, green triangle, blue circle, blue triangle, red square, green square, green circle, blue square. If I draw a straight line between the red triangle and the green triangle, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: red circle\n\nQuestion: In a 3x3 grid there are the following objects, listed left to right, top to bottom: blue triangle, red circle, red square, green circle, red triangle, blue circle, green triangle, blue square, green square. If I draw a straight line between the blue triangle and the green square, what shapes, if any, does that line intersect? The shapes where the line start and end do not count as intersecting the line. If a shape is intersected, return the shape.    \nAnswer: blue circle\n\nQuestion: In a 3x

# Letter counting task
In this task, a number of words is generated with numbers and letters in them. The task is to count the number of letters (excluding numbers, explicitly) in total that are present in the provided list.

In [136]:
import string
alphabetic_letters=set(string.ascii_lowercase)

In [137]:
def generate_list_words(list_len:int)->list[str]:
    def generate_single_word():
        # Let's make it between 3 and 6 chars long
        word_len=random.choice([3,4,5,6])
        word=""
        for _ in range(word_len):
            is_letter=random.choice([False,True])
            if is_letter:
                word+=random.choice(string.ascii_lowercase)
            else:
                word+=random.choice(string.digits)
        return word
        
    return [generate_single_word() for i in range(list_len)]
print(generate_list_words(3))
    

['h79mb', 'btx4h2', 'ma10pd']


In [143]:
def letters_in_word(word:str)->int:
    return len(list(filter(lambda char: char in alphabetic_letters,word)))

def get_prompt_and_answer(words:list[str]):
    letters=sum([letters_in_word(w) for w in words])
    wordlist=', '.join(words)
    prompt=f"""Question: What is the total number of letters are in the words in the list?
    Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters.
    The word list is: {wordlist}.
    Reason step by step. First, calculate and write how many letters are in each word, then sum them.
    
    \n\nAnswer:
    """.replace("\n    "," ")
    return prompt,letters
prompt=get_prompt_and_answer(generate_list_words(3))
print(prompt)
await call_model(prompt[0])

('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: mre7l5, up72, dx4477. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: ', 8)


'There are a total of 12 letters in the words in the list.'

In [188]:
N_SAMPLES=50
TEMP=0
def check_answer(result:str,answer:int):
            # Get all numbers
            nums=[int(s) for s in result.split() if s.isdigit()]
            if len(nums)<1:
                raise Exception(f"Check failed for result: {result}")
            if nums[-1]==answer:
                return True
            else:
                return False
# Assumes the number is the last part of the prompt
async def eval_model(prompt_and_answer:tuple[str,int]):
    prompt=prompt_and_answer[0]
    model_result=await call_model(prompt,1000,TEMP)
    is_model_correct=check_answer(model_result,prompt_and_answer[1])
    return prompt,model_result,is_model_correct
await eval_model(get_prompt_and_answer(generate_list_words(3)))

('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: 2ura, am2629, f69. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: ',
 '2ura: 4 letters\nam2629: 5 letters\nf69: 2 letters\n\nTotal: 11 letters',
 False)

## Three words

In [150]:
# Zero-shot
prompt=lambda:get_prompt_and_answer(generate_list_words(3))
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(prompt()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[]

In [165]:
# One-shot
def get_example():
    prompt,answer=get_prompt_and_answer(generate_list_words(3))
    example=f"{prompt}{answer}\n"
    question=get_prompt_and_answer(generate_list_words(3))
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[]

In [189]:
# Two-shot; 10%
def get_example():
    prompt,answer=get_prompt_and_answer(generate_list_words(3))
    prompt2,answer2=get_prompt_and_answer(generate_list_words(3))
    example1=f"{prompt}{answer}\n"
    example2=f"{prompt2}{answer2}\n"
    example=example1+example2
    question=get_prompt_and_answer(generate_list_words(3))
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
len(list(filter(lambda x: x[2],result)))


5

In [192]:
# Three shot; 8%
def get_example():
    prompt,answer=get_prompt_and_answer(generate_list_words(3))
    prompt2,answer2=get_prompt_and_answer(generate_list_words(3))
    prompt3,answer3=get_prompt_and_answer(generate_list_words(3))
    example1=f"{prompt}{answer}\n"
    example2=f"{prompt2}{answer2}\n"
    example3=f"{prompt3}{answer3}\n"
    example=example1+example2+example3
    question=get_prompt_and_answer(generate_list_words(3))
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
len(list(filter(lambda x: x[2],result)))

4

## Four words


In [194]:
# Zero-shot 3/50
prompt=lambda:get_prompt_and_answer(generate_list_words(4))
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(prompt()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: 2nmj, z77m, t74n, 9555k. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: ',
  '2nmj: 4 letters\n\nz77m: 2 letters\n\nt74n: 2 letters\n\n9555k: 0 letters\n\nTotal: 8 letters',
  True),
 ('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: hah, lqh, quni43, 46dp. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: ',
  'There are a total of 12 letters in the words in the list.',
  True),
 ('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only le

In [195]:
# One-shot 3/50
def get_example():
    prompt,answer=get_prompt_and_answer(generate_list_words(4))
    example=f"{prompt}{answer}\n"
    question=get_prompt_and_answer(generate_list_words(3))
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
list(filter(lambda x: x[2],result))

[('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: 09wc6, ia4, zl74, 33pyyb. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: 10\n\nQuestion: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: 0lp, 75t98, ir39by. Reason step by step. First, calculate and write how many letters are in each word, then sum them.  \n\nAnswer: ',
  'There are a total of 7 letters in the words in the list.',
  True),
 ('Question: What is the total number of letters are in the words in the list? Do not count the numeric characters, count only letters in the alphabet. E.g. the word blap123 has 4 letters. The word list is: dthz2, 93xy, 11wp5, 5m369u. Reason

In [196]:
# Two-shot; 4/50
def get_example():
    prompt,answer=get_prompt_and_answer(generate_list_words(4))
    prompt2,answer2=get_prompt_and_answer(generate_list_words(4))
    example1=f"{prompt}{answer}\n"
    example2=f"{prompt2}{answer2}\n"
    example=example1+example2
    question=get_prompt_and_answer(generate_list_words(4))
    return example+"\n"+question[0],question[1]
with ThreadPoolExecutor(max_workers=50) as p:
    tasks=[eval_model(get_example()) for _ in range(N_SAMPLES)]
    result=await asyncio.gather(*tasks)
len(list(filter(lambda x: x[2],result)))

4